# Ratterdam Linear Models
## Linear Mixed Effects Models, using Generalized framework if needed. Oct 2020. 

### Imports and global settings

In [1]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
from scipy.stats import sem, binom_test, shapiro, bartlett
import utility_fx as util
import os
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import ratterdam_DataFiltering as Filt
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from scipy.stats import kruskal
from collections import OrderedDict
from statsmodels.genmod.bayes_mixed_glm import PoissonBayesMixedGLM as PBGLMER

In [2]:
%qtconsole --style native
%matplotlib qt5

### Load Data

In [4]:
rat = 'R808'
expCode = 'BRD7'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
savepath = f'E:\\Ratterdam\\{rat}\\anovas\\'

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
clustlist = util.getClustList(datafile)
population = OrderedDict()

for clust in clustlist:
    unit = Core.UnitData(clust, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
    unit.loadData_raw()
    validalleys = []
    valid, acorr, alleys = util.checkInclusion(unit, 1)
    if valid:
        print(clust)
        unit.acorr = acorr
        unit.validAlleys = alleys
        population[clust] = unit

E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:178: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:178: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:184: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:184: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


TT1\cl-maze1.1
TT1\cl-maze1.2
TT1\cl-maze1.3
TT14\cl-maze1.2
TT14\cl-maze1.3
TT15\cl-maze1.1
TT15\cl-maze1.5
TT15\cl-maze1.6
TT3\cl-maze1.2
TT6\cl-maze1.1
TT6\cl-maze1.3
TT6\cl-maze1.4
TT9\cl-maze1.1


### Create "Long Form" of Data. Single Day
#### Nested Structure: Cell > trial > texture > reward > spatial bin. 

In [6]:
cells, cellNames, trials, textures, rewards, spatialBins, alleys, firingRate = [], [], [], [], [], [], [], []
nbins = Def.singleAlleyBins[0]-1
for cellID, (unitname, unit) in enumerate(population.items()):
    print(unitname)
    for alley in unit.validAlleys:
        for i,visit in enumerate(unit.alleys[alley]):
            alleys.extend([alley]*nbins)
            cells.extend([cellID]*nbins)
            cellNames.extend([unitname]*nbins)
            trials.extend([i]*nbins)
            txt = visit['metadata']['stimulus']
            textures.extend([txt]*nbins)
            rewards.extend([visit['metadata']['reward']]*nbins)
            spatialBins.extend(range(12))
            firingRate.extend(visit['ratemap1d'])
            
firingRate = np.log(np.asarray(firingRate)+1)
data = {'rate':firingRate, 'cell':cells, 'name':cellNames, 'alley': alleys, 'trial':trials, 'texture':textures, 'reward':rewards, 'spatialBin':spatialBins}
alldata = pd.DataFrame(data=data)
alldata.dropna(inplace=True) 

TT1\cl-maze1.1
TT1\cl-maze1.2
TT1\cl-maze1.3
TT14\cl-maze1.2
TT14\cl-maze1.3
TT15\cl-maze1.1
TT15\cl-maze1.5
TT15\cl-maze1.6
TT3\cl-maze1.2
TT6\cl-maze1.1
TT6\cl-maze1.3
TT6\cl-maze1.4
TT9\cl-maze1.1


In [11]:
alldata.to_csv(f"E:\\Ratterdam\\R_data\\{rat}{expCode}.csv", header=True, index=False)

In [10]:
Def.beltwayAlleyLookup

{1: 4, 3: 3, 5: 5, 7: 6, 8: 7, 10: 8, 11: 9, 16: 1, 17: 2}

### LMER On All Cells in A Day

In [126]:
df = alldata
df['group'] = 1
vcf = {"cell":          "1 + C(cell)",
       "trial":         "1 + trial",
       "spatialBin":    "1 + spatialBin",
       "reward":        "1 + C(reward)",
        "alley":        "1 + alley"
        }

model = sm.MixedLM.from_formula("rate ~ C(texture)", groups=df["group"],                                                    
                                vc_formula=vcf, data=df)
result = model.fit()
print(result.summary())

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2133: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


          Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: rate       
No. Observations:  29636   Method:             REML       
No. Groups:        1       Scale:              0.6465     
Min. group size:   29636   Log-Likelihood:     -35683.2752
Max. group size:   29636   Converged:          Yes        
Mean group size:   29636.0                                
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept        1.040    0.549  1.895 0.058 -0.036  2.116
C(texture)[T.B] -0.012    0.011 -1.014 0.311 -0.034  0.011
C(texture)[T.C] -0.002    0.011 -0.148 0.882 -0.024  0.021
alley Var        0.002                                    
cell Var         0.293    0.064                           
reward Var       0.002                                    
spatialBin Var   0.002                                    
trial Va

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2189: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


### LMER On Cells Separately

In [215]:
results = {}
for i,cell in enumerate(population.keys()):
    print(cell)
    df = alldata.loc[alldata['cell'] == i]
    df['group'] = 1
    vcf = {#"cell":          "1 + C(cell)",
           "trial":         "1 + trial",
           #"spatialBin":    "1 + spatialBin",
           "reward":        "1 + C(reward)",
            #"alley":        "1 + alley"
            }

    model = sm.MixedLM.from_formula("rate ~ C(texture)*spatialBin*alley", groups=df["group"],                                                    
                                    vc_formula=vcf, data=df)
    result = model.fit()
    print(result.summary())
    results[f"{cell}"] = result

TT11\cl-maze1.1


C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2133: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space

                   Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       rate      
No. Observations:        3171          Method:                   REML      
No. Groups:              1             Scale:                    1.0043    
Min. group size:         3171          Log-Likelihood:           -4551.1867
Max. group size:         3171          Converged:                Yes       
Mean group size:         3171.0                                            
---------------------------------------------------------------------------
                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                         0.947    0.123  7.731 0.000  0.707  1.187
C(texture)[T.B]                   0.031    0.160  0.196 0.845 -0.282  0.344
C(texture)[T.C]                  -0.388    0.161 -2.414 0.016 -0.702 -0.073
spatialBin                     

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2133: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2133: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to conver

                   Mixed Linear Model Regression Results
Model:                    MixedLM       Dependent Variable:       rate     
No. Observations:         698           Method:                   REML     
No. Groups:               1             Scale:                    0.2395   
Min. group size:          698           Log-Likelihood:           -524.6968
Max. group size:          698           Converged:                No       
Mean group size:          698.0                                            
---------------------------------------------------------------------------
                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                         8.445    1.273  6.633 0.000  5.950 10.941
C(texture)[T.B]                  -1.663    1.805 -0.921 0.357 -5.201  1.875
C(texture)[T.C]                  -1.748    1.790 -0.977 0.329 -5.255  1.760
spatialBin                     

...

                   Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       rate      
No. Observations:        3171          Method:                   REML      
No. Groups:              1             Scale:                    0.3890    
Min. group size:         3171          Log-Likelihood:           -3053.3065
Max. group size:         3171          Converged:                No        
Mean group size:         3171.0                                            
---------------------------------------------------------------------------
                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                         2.639    0.077 34.395 0.000  2.489  2.790
C(texture)[T.B]                   0.317    0.100  3.181 0.001  0.121  0.512
C(texture)[T.C]                   0.108    0.100  1.076 0.282 -0.088  0.303
spatialBin                     

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu


TT3\cl-maze1.4
                   Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       rate      
No. Observations:        1397          Method:                   REML      
No. Groups:              1             Scale:                    0.6515    
Min. group size:         1397          Log-Likelihood:           -1720.4641
Max. group size:         1397          Converged:                Yes       
Mean group size:         1397.0                                            
---------------------------------------------------------------------------
                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                         1.269    0.175  7.238 0.000  0.925  1.613
C(texture)[T.B]                  -0.516    0.201 -2.565 0.010 -0.910 -0.122
C(texture)[T.C]                  -0.559    0.202 -2.761 0.006 -0.955 -0.162
spatialBin     

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2133: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2189: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launc

LinAlgError: Singular matrix

### LMER On Alleys Separately

In [268]:
# By Alley Separately
results = {}
for i,cell in enumerate(population.keys()):
    alleys = population[cell].validAlleys
    print(cell)
    for alley in alleys:
        df = alldata.loc[(alldata['cell'] == i) & (alldata['alley']==alley)]
        df['group'] = 1
        vcf = {#"cell":          "1 + C(cell)",
               "trial":         "1 + trial",
               "spatialBin":    "1 + spatialBin",
               "reward":        "1 + C(reward)",
                #"alley":        "1 + alley"
                }

        model = sm.MixedLM.from_formula("rate ~ C(texture)*spatialBin", groups=df["group"],                                                    
                                        vc_formula=vcf, data=df)
        result = model.fit()
        print(result.summary())
        results[f"{cell}_{alley}"] = result

TT11\cl-maze1.1
                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     rate     
No. Observations:       360         Method:                 REML     
No. Groups:             1           Scale:                  0.7495   
Min. group size:        360         Log-Likelihood:         -471.0821
Max. group size:        360         Converged:              Yes      
Mean group size:        360.0                                        
---------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                   1.153    0.870  1.325 0.185 -0.552  2.857
C(texture)[T.B]             0.132    0.221  0.596 0.551 -0.301  0.565
C(texture)[T.C]             0.051    0.212  0.242 0.809 -0.363  0.466
spatialBin                  0.064    0.177  0.360 0.719 -0.284  0.411
C(texture)[T.B]:spat

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2189: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarnin

                  Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       rate     
No. Observations:       357           Method:                   REML     
No. Groups:             1             Scale:                    0.6317   
Min. group size:        357           Log-Likelihood:           -440.9488
Max. group size:        357           Converged:                Yes      
Mean group size:        357.0                                            
-------------------------------------------------------------------------
                           Coef.    Std.Err.     z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                   0.606        0.983  0.616 0.538 -1.322  2.533
C(texture)[T.B]            -0.234        0.193 -1.212 0.225 -0.612  0.144
C(texture)[T.C]            -0.095        0.199 -0.475 0.635 -0.485  0.296
spatialBin                 -0.070        0.795 -0.088 0.

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

                  Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       rate     
No. Observations:       334           Method:                   REML     
No. Groups:             1             Scale:                    0.6893   
Min. group size:        334           Log-Likelihood:           -422.3378
Max. group size:        334           Converged:                Yes      
Mean group size:        334.0                                            
-------------------------------------------------------------------------
                           Coef.    Std.Err.     z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                   2.934        0.847  3.462 0.001  1.273  4.595
C(texture)[T.B]            -0.022        0.207 -0.107 0.915 -0.427  0.383
C(texture)[T.C]            -0.415        0.221 -1.881 0.060 -0.847  0.017
spatialBin                 -0.184        0.831 -0.222 0.

...

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2133: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2189: ConvergenceWarning: The Hessian matrix at the estimated parameter values is 


                Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     rate     
No. Observations:       339         Method:                 REML     
No. Groups:             1           Scale:                  0.4924   
Min. group size:        339         Log-Likelihood:         -376.8353
Max. group size:        339         Converged:              Yes      
Mean group size:        339.0                                        
---------------------------------------------------------------------
                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                   0.428    0.776  0.552 0.581 -1.093  1.949
C(texture)[T.B]             0.051    0.178  0.284 0.776 -0.298  0.400
C(texture)[T.C]             0.142    0.175  0.812 0.417 -0.201  0.485
spatialBin                  0.071    0.702  0.101 0.920 -1.305  1.447
C(texture)[T.B]:spatialBin -0.026  

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2133: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

                  Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       rate     
No. Observations:       352           Method:                   REML     
No. Groups:             1             Scale:                    0.2641   
Min. group size:        352           Log-Likelihood:           -279.7199
Max. group size:        352           Converged:                Yes      
Mean group size:        352.0                                            
-------------------------------------------------------------------------
                           Coef.    Std.Err.     z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                   0.246        0.524  0.470 0.638 -0.780  1.273
C(texture)[T.B]             0.005        0.126  0.043 0.965 -0.241  0.252
C(texture)[T.C]            -0.011        0.130 -0.083 0.934 -0.266  0.244
spatialBin                 -0.018        0.514 -0.035 0.

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2168: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2189: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


## Poisson Bayes Generalized Mixed Effects - All Cells in A Day

In [163]:
df = alldata
vcf = {"cell":          "1 + C(cell)",
       "trial":         "1 + trial",
       "spatialBin":    "1 + spatialBin",
       "reward":        "1 + C(reward)",
        "alley":        "1 + alley"
        }
rate = np.asarray(df['rate'])

txt = []
for row in df['texture']:
    if row == 'A':
        txt.append(0)
    elif row == 'B':
        txt.append(1)
    elif row == 'C':
        txt.append(2)
txt = np.asarray(txt)

cell = np.asarray(df['cell'])
alley = np.asarray(df['alley'])
trial = np.asarray(df['trial'])
reward = np.asarray(df['reward'])
spatialBin = np.asarray(df['spatialBin'])

model = PBGLMER(rate, txt, np.column_stack((cell,alley,trial,reward,spatialBin)), [1,2,3,4,5])
result = model.fit_vb()
print(result.summary())

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:780: RuntimeWarning: overflow encountered in exp
  return -self.vb_elbo(x[:n], np.exp(x[n:]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:625: RuntimeWarning: divide by zero encountered in log
  r = (iv + np.sum(np.log(fep_sd)) + np.sum(np.log(vcp_sd)) + np.sum(
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:626: RuntimeWarning: divide by zero encountered in log
  np.log(vc_sd)))
C:\Users\whockei1\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:784: RuntimeWarning: overflow encountered in exp
  gm, gs = self.vb_elb

                                                   Poisson Mixed GLM Results
     Type           Post. Mean                                       Post. SD                                SD  SD (LB) SD (UB)
--------------------------------------------------------------------------------------------------------------------------------
FE_1    M -27325399408196548820992.0000                                                             0.0000                      
VC_1    V                        0.0000                                                                inf 1.000   0.000     inf
VC_2    V                     -624.1953                                                                inf 0.000   0.000     inf
VC_3    V                     -537.1353                                                                inf 0.000   0.000     inf
VC_4    V                     -596.7500                                                                inf 0.000   0.000     inf
VC_5    V           

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:814: RuntimeWarning: overflow encountered in square
  iv -= np.sum((vc_mean**2 + vc_sd**2) * np.exp(2 * (s**2 - m))) / 2
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:818: RuntimeWarning: overflow encountered in square
  iv -= 0.5 * (vcp_mean**2 + vcp_sd**2).sum() / self.vcp_p**2
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:784: RuntimeWarning: overflow encountered in exp
  gm, gs = self.vb_elbo_grad(x[:n], np.exp(x[n:]))
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:649: RuntimeWarning: invalid value encountered in true_divide
  r = u / np.sqrt(tv)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\bayes_mixed_glm.py:832: RuntimeWarning: overflow encountered